In [ ]:
import csv
import re
from crypt import crypt


def create_zdrobnenina(name):
    # Pravidlá pre tvorbu zdrobnelín v slovenčine
    
    # Špeciálne prípady
    special_cases = {
        'Jozef': 'Jožko', 
        'Michal': 'Miško', 
        'Peter': 'Peťko',
        'Pavol': 'Paľko',
        'Tomáš': 'Tomáško',
        'Jakub': 'Kubko',
        'Adam': 'Adamko',
        'Daniel': 'Danko',
        'Martin': 'Martinko'
    }
    
    if name in special_cases:
        return special_cases[name]
    
    # Všeobecné pravidlá
    # Najprv skúsime odstrániť koncovky a pridať typické zdrobnelinové
    name = name.strip()
    
    # Pre ženské mená končiace na -a
    if name.endswith('a'):
        # Špeciálne prípady pre ženské mená
        if name == 'Zuzana': return 'Zuzka'
        if name == 'Katarína': return 'Katka'
        if name == 'Lucia': return 'Lucka'
        
        # Všeobecné pravidlo pre ženské mená končiace na -a
        return name[:-1] + 'ka'
    
    # Pre mužské mená končiace na spoluhlásku
    if re.search(r'[konrtlmpdvž]$', name, re.IGNORECASE):
        # Špeciálne prípady pre mužské mená
        if name == 'David': return 'Davidko'
        
        # Všeobecné pravidlo pre mužské mená
        return name + 'ko'
    
    # Pre mená končiace na -el, -er
    if name.endswith('el') or name.endswith('er'):
        return name + 'ko'
    
    # Ak žiadne pravidlo nezabralo, pridáme jednoducho -ko
    return name + 'ko'

def load_names_from_csv(filename):
    names = []
    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Preskočenie hlavičky, ak existuje
        for row in reader:
            names.append(row[1])
    return names

def generate_zdrobneniny(filename):
    names = load_names_from_csv(filename)
    names_with_zdrobneniny = []

    for name in names:
        zdrobnenina = create_zdrobnenina(name)
        names_with_zdrobneniny.extend([name.lower(), zdrobnenina.lower()])
    
    return names_with_zdrobneniny

# Príklad použitia
if __name__ == "__main__":
    filename = 'sk-meniny.csv'
    zdrobneniny = generate_zdrobneniny(filename)

In [31]:
from base64 import b64encode
from hashlib import md5
import itertools

# Function to compute hash
def crypt(passwd, salt):
    m = md5()
    m.update(passwd.encode('utf-8'))
    m.update(salt.encode('utf-8'))
    return b64encode(m.digest()).decode('utf-8')

# Function to load shadow file
def load_shadow_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    data = [line.strip().split(":") for line in lines]
    return data  # List in the format [login, salt, hash]

def generate_names_variants_lazy(names):
    for name in names:
        name = name.lower()
        # Yield variations with capitalized letters
        for i in range(len(name)):
            yield name[:i] + name[i].upper() + name[i+1:]
        # Yield diminutives (if the name ends in 'a')
        if name.endswith("a"):
            yield name[:-1] + "ka"

# Lazy password generator for category 2: 6-7 lowercase letters
def generate_small_letter_passwords_lazy():
    chars = "abcdefghijklmnopqrstuvwxyz"
    for length in range(6, 8):  # Lengths 6 and 7
        for password in itertools.product(chars, repeat=length):
            yield ''.join(password)

# Lazy password generator for category 3: 4-5 mixed characters
def generate_mixed_passwords_lazy():
    chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    for length in range(4, 6):  # Lengths 4 and 5
        for password in itertools.product(chars, repeat=length):
            yield ''.join(password)

def crack_passwords(filename):
    shadow_data = load_shadow_file(filename)
    for login, salt, hash_value in shadow_data:
        # Shared generator for all categories (lazy generation)
        password_generator = itertools.chain(
            generate_names_variants_lazy(zdrobneniny),
            # generate_small_letter_passwords_lazy(),
            # generate_mixed_passwords_lazy(),
        )

        for password in password_generator:
            hash = crypt(password, salt)
            if hash == hash_value:
                print(f"Found password for {login}: {password}")

# Run the cracking function for all files
for i in range(1, 5):  # shadow1.txt to shadow4.txt
    filename = f"shadow{i}.txt"
    cracked_passwords = crack_passwords(filename)
    print(f"Cracked passwords from {filename}:")
    if cracked_passwords:
        for login, password in cracked_passwords:
            print(f"  Login: {login}, Password: {password}")

Found password for krajciko: katkA
Found password for nemecm: milaDa
Cracked passwords from shadow1.txt:
Found password for kalous: adamKo
Cracked passwords from shadow2.txt:
Found password for soukal: adamKo
Cracked passwords from shadow3.txt:
Found password for petran: leNka
Found password for sagan00: adamKo
Cracked passwords from shadow4.txt:


In [36]:
from base64 import b64encode
from hashlib import md5
import itertools

# Function to compute hash
def crypt(passwd, salt):
    m = md5()
    m.update(passwd.encode('utf-8'))
    m.update(salt.encode('utf-8'))
    return b64encode(m.digest()).decode('utf-8')

# Function to load shadow file
def load_shadow_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    data = [line.strip().split(":") for line in lines]
    return data  # List in the format [login, salt, hash]

def generate_names_variants_lazy(names):
    for name in names:
        name = name.lower()
        # Yield variations with capitalized letters
        for i in range(len(name)):
            yield name[:i] + name[i].upper() + name[i+1:]
        # Yield diminutives (if the name ends in 'a')
        if name.endswith("a"):
            yield name[:-1] + "ka"

# Lazy password generator for category 2: 6-7 lowercase letters
def generate_small_letter_passwords_lazy():
    chars = "abcdefghijklmnopqrstuvwxyz"
    for length in range(6, 8):  # Lengths 6 and 7
        for password in itertools.product(chars, repeat=length):
            yield ''.join(password)

# Lazy password generator for category 3: 4-5 mixed characters
def generate_mixed_passwords_lazy():
    chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    for length in range(4, 6):  # Lengths 4 and 5
        for password in itertools.product(chars, repeat=length):
            yield ''.join(password)

def crack_passwords(filename):
    index = 0
    shadow_data = load_shadow_file(filename)
    for login, salt, hash_value in shadow_data:
        # Shared generator for all categories (lazy generation)
        password_generator = itertools.chain(
            # generate_names_variants_lazy(zdrobneniny),
            # generate_small_letter_passwords_lazy(),
            generate_mixed_passwords_lazy(),
        )

        for password in password_generator:
            hash = crypt(password, salt)
            if hash == hash_value:
                print(f"Found password for {login}: {password}")
                break
                
        print(f'Login {login} done. {index} / {len(shadow_data)}')
        index += 1

# Run the cracking function for all files
for i in range(1, 5):  # shadow1.txt to shadow4.txt
    filename = f"shadow{i}.txt"
    print(f"Cracking from {filename}:")
    cracked_passwords = crack_passwords(filename)
    print(f"Cracked passwords from {filename}:")
    if cracked_passwords:
        for login, password in cracked_passwords:
            print(f"  Login: {login}, Password: {password}")

Cracking from shadow1.txt:
Login stepanov done. 0 / 91
Login rychnavs done. 1 / 91
Login glemba done. 2 / 91
Login kubik done. 3 / 91
Login fratrikova7 done. 4 / 91
Login hajtinger done. 5 / 91
Found password for kochlan4: petkO
Login kochlan4 done. 6 / 91
Login franek17 done. 7 / 91
Login balak done. 8 / 91
Login kopacova done. 9 / 91
Found password for pallova: s7xG
Login pallova done. 10 / 91
Login urbanico done. 11 / 91
Login kalous done. 12 / 91
Login urban14 done. 13 / 91
Login ecker done. 14 / 91
Login sagan00 done. 15 / 91
Login turoci2 done. 16 / 91
Login zuscakv done. 17 / 91
Login durinat done. 18 / 91
Login szabo22 done. 19 / 91
Login pitak3 done. 20 / 91
Login visnovsl done. 21 / 91
Login burgel done. 22 / 91
Login kontroso done. 23 / 91
Login vrab5 done. 24 / 91
Login slavikm done. 25 / 91


KeyboardInterrupt: 